https://medium.com/agents-and-robots/the-bitter-truth-python-3-11-vs-cython-vs-c-performance-for-simulations-babc85cdfef5

In [1]:
from juliacall import Main as jl
%load_ext juliacall.ipython
# JuliaCall comes with its own Julia dependency file juliapkg.json
# however for binder it is much simpler to just reuse binder's installation mechanism
%julia Pkg.activate(Base.current_project())
%julia using PythonCall
%julia set_var(k, v) = @eval $(Symbol(k)) = $v

  Activating project at `~/Projects/Jolin.io/workshop-accelerate-Python-with-Julia`


set_var (generic function with 1 method)

In [2]:
import math
import random

from datetime import datetime
random.seed(datetime.now().timestamp())

WORLD_WIDTH = 2560
WORLD_HEIGHT = 1440
TIMESTEPS = 10000

jl.set_var("WORLD_WIDTH", WORLD_WIDTH)
jl.set_var("WORLD_HEIGHT", WORLD_HEIGHT)
jl.set_var("TIMESTEPS", TIMESTEPS)

10000

In [3]:
class PythonAgent():
    def __init__(self, x=None, y=None, world_width=0, world_height=0):
        super().__init__()

        # default values
        self.vmax = 2.0

        # initial position
        self.world_width = world_width
        self.world_height = world_height
        self.x = x if x else random.randint(0, self.world_width)
        self.y = y if y else random.randint(0, self.world_height)

        # initial velocity
        self.dx = 0
        self.dy = 0

        # inital values
        self.is_alive = True
        self.target = None
        self.age = 0
        self.energy = 0

    def update(self, food=()):
        self.age = self.age + 1

        # we can't move
        if self.vmax == 0:
            return

        # target is dead, don't chase it further
        if self.target and not self.target.is_alive:
            self.target = None

        # eat the target if close enough
        if self.target:
            squared_dist = (self.x - self.target.x) ** 2 + (self.y - self.target.y) ** 2
            if squared_dist < 400:
                self.target.is_alive = False
                self.energy = self.energy + 1

        # agent doesn't have a target, find a new one
        if not self.target:
            min_dist = 9999999
            min_agent = None
            for a in food:
                if a is not self and a.is_alive:
                    sq_dist = (self.x - a.x) ** 2 + (self.y - a.y) ** 2
                    if sq_dist < min_dist:
                        min_dist = sq_dist
                        min_agent = a
            if min_dist < 100000:
                self.target = min_agent

        # initalize 'forces' to zero
        fx = 0
        fy = 0

        # move in the direction of the target, if any
        if self.target:
            fx += 0.1*(self.target.x - self.x)
            fy += 0.1*(self.target.y - self.y)

        # update our direction based on the 'force'
        self.dx = self.dx + 0.05 * fx
        self.dy = self.dy + 0.05 * fy

        # slow down agent if it moves faster than it max velocity
        velocity = math.sqrt(self.dx ** 2 + self.dy ** 2)
        if velocity > self.vmax:
            self.dx = (self.dx / velocity) * (self.vmax)
            self.dy = (self.dy / velocity) * (self.vmax)

        # update position based on delta x/y
        self.x = self.x + self.dx
        self.y = self.y + self.dy

        # ensure it stays within the world boundaries
        self.x = max(self.x, 0)
        self.x = min(self.x, self.world_width)
        self.y = max(self.y, 0)
        self.y = min(self.y, self.world_height)

In [4]:
from agent import Agent as CythonAgent

In [5]:
import agent

In [6]:
agent.main(WORLD_WIDTH=WORLD_WIDTH, WORLD_HEIGHT=WORLD_HEIGHT, TIMESTEPS=TIMESTEPS)

AttributeError: 'agent.Prey' object has no attribute 'is_alive'

In [132]:
def main(Agent):

    class Predator(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 2.5

    class Prey(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 2.0

    class Plant(Agent):
        def __init__(self, **kwargs):
            super().__init__(**kwargs)
            self.vmax = 0.0

            
    # open the ouput file
    # f = open('output.csv', 'w')
    # print(0, ',', 'Title', ',', 'Predator Prey Relationship / Example 02 / Cython', file=f)

    # create initial agents
    kwargs = dict(
        world_width=WORLD_WIDTH,
        world_height=WORLD_HEIGHT,
    )
    preys = [Prey(**kwargs) for i in range(10)]
    predators = [Predator(**kwargs) for i in range(10)]
    plants = [Plant(**kwargs) for i in range(100)]

    timestep = 0
    while timestep < TIMESTEPS:
        # update all agents
        #[f.update([]) for f in plants]  # no need to update the plants; they do not move
        [a.update(plants) for a in preys]
        [a.update(preys) for a in predators]

        # handle eaten and create new plant
        plants = [p for p in plants if p.is_alive is True]
        plants = plants + [Plant(**kwargs) for i in range(2)]

        # handle eaten and create new preys
        preys = [p for p in preys if p.is_alive is True]

        for p in preys[:]:
            if p.energy > 5:
                p.energy = 0
                preys.append(Prey(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20), **kwargs))

        # handle old and create new predators
        predators = [p for p in predators if p.age < 2000]

        for p in predators[:]:
            if p.energy > 10:
                p.energy = 0
                predators.append(Predator(x = p.x + random.randint(-20, 20), y = p.y + random.randint(-20, 20), **kwargs))

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep = timestep + 1

    print(len(predators), len(preys), len(plants))

In [133]:
%timeit -r 3 main(PythonAgent)

0 0 20036
24 1040 338
0 2393 343
0 1 20044
The slowest run took 16.77 times longer than the fastest. This could mean that an intermediate result is being cached.
1min 10s ± 49.2 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [14]:
%timeit main(CythonAgent)

10 2 2043
10 60 1583
10 16 1895
10 44 1611
10 31 1730
10 2 2049
10 2 2003
10 9 1899
10 2 2018
10 26 1769
10 0 1950
10 16 1879
10 40 1741
10 35 1779
10 28 1766
10 26 1806
10 4 2029
10 31 1740
10 32 1680
10 1 1999
10 20 1775
10 20 1893
10 21 1803
11 11 1851
10 31 1800
10 0 2031
10 11 1936
10 21 1847
10 9 1878
10 36 1616
10 10 1909
10 2 2019
10 23 1913
10 8 1919
10 12 1778
10 16 1917
10 4 1943
11 11 1902
10 16 1909
10 21 1845
10 30 1666
10 0 2056
10 29 1779
10 71 1456
10 20 1890
10 48 1571
10 0 1979
10 12 1956
10 30 1734
10 24 1871
10 44 1640
10 27 1821
10 1 1977
11 13 1872
10 0 1999
10 19 1857
10 44 1701
10 2 2027
10 26 1857
10 57 1552
10 7 1974
10 47 1645
10 29 1806
10 24 1827
10 47 1688
10 18 1819
10 12 1987
10 38 1753
10 34 1707
10 23 1888
10 53 1657
10 1 2004
10 18 1750
10 27 1700
10 8 1924
10 11 1874
10 21 1842
10 12 1987
10 16 1878
10 3 2003
10 5 1979
116 ms ± 7.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# C++

In [ ]:
!g++ example.cpp -o example -std=c++11 -O3

In [20]:
!perf stat -r 100 -ddd ./example $TIMESTEPS

ok 0, 1304, 93
ok 0, 1304, 93
ok 0, 1306, 82
ok 0, 1306, 82
ok 0, 1306, 82
ok 0, 1302, 80
ok 0, 1302, 80
ok 0, 1300, 90
ok 0, 1300, 90
ok 26, 266, 562
ok 26, 266, 562
ok 26, 266, 562
ok 0, 1306, 86
ok 0, 1306, 86
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 0, 20049
ok 0, 1290, 73
ok 0, 1290, 73
ok 0, 1290, 73
ok 0, 1291, 101
ok 0, 1291, 101
ok 0, 0, 19960
ok 0, 0, 19960
ok 0, 0, 19960
ok 0, 0, 19960
ok 0, 0, 19960
ok 0, 0, 19960
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 0, 19996
ok 0, 1301, 80
ok 0, 1301, 80
ok 14, 578, 228
ok 14, 578, 228
ok 14, 578, 228
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19970
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 0, 19992
ok 0, 1286, 77
ok 0, 1286, 77
ok 0, 1289, 85
ok 0, 1289, 85
ok 0, 1289, 85
ok

In [119]:
%%julia
Base.@kwdef mutable struct Agent
    vmax::Float64 = 2.5
    world_width::Int = 10
    world_height::Int = 10
    x::Int = rand(0:world_width)
    y::Int = rand(0:world_height)
    
    # initial velocity
    dx::Float64 = 0.0
    dy::Float64 = 0.0

    # inital values
    is_alive::Bool = true
    target::Union{Nothing, Agent} = nothing
    age::Int = 0
    energy::Int = 0
end

Predator(; kwargs...) = Agent(; vmax = 2.5, kwargs...)
Prey(; kwargs...) = Agent(; vmax = 2.0, kwargs...)
Plant(; kwargs...) = Agent(; vmax = 0.0, kwargs...)

Plant (generic function with 1 method)

In [120]:
%%julia
function update!(self::Agent, food::Vector{Agent})
    self.age = self.age + 1

    # we can't move
    if self.vmax == 0.0
        return
    end

    # target is dead, don't chase it further
    if self.target !== nothing && !self.target.is_alive
        self.target = nothing
    end
    
    # eat the target if close enough
    if self.target !== nothing
        squared_dist = (self.x - self.target.x) ^ 2 + (self.y - self.target.y) ^ 2
        if squared_dist < 400
            self.target.is_alive = false
            self.energy = self.energy + 1
        end
    # agent doesn't have a target, find a new one
    else
        min_dist = 9999999
        min_agent = nothing
        for a in food
            if a !== self && a.is_alive
                sq_dist = (self.x - a.x) ^ 2 + (self.y - a.y) ^ 2
                if sq_dist < min_dist
                    min_dist = sq_dist
                    min_agent = a
                end
            end
        end
        if min_dist < 100000
            self.target = min_agent
        end
    end

    # initalize 'forces' to zero
    fx = 0.0
    fy = 0.0

    # move in the direction of the target, if any
    if self.target !== nothing
        fx += 0.1 * (self.target.x - self.x)
        fy += 0.1 * (self.target.y - self.y)
    end

    # update our direction based on the 'force'
    self.dx = self.dx + 0.05 * fx
    self.dy = self.dy + 0.05 * fy

    # slow down agent if it moves faster than it max velocity
    velocity = sqrt(self.dx ^ 2 + self.dy ^ 2)
    if velocity > self.vmax
        self.dx = (self.dx / velocity) * (self.vmax)
        self.dy = (self.dy / velocity) * (self.vmax)
    end

    # update position based on delta x/y
    self.x = self.x + Int(round(self.dx))
    self.y = self.y + Int(round(self.dy))

    # ensure it stays within the world boundaries
    self.x = max(self.x, 0)
    self.x = min(self.x, self.world_width)
    self.y = max(self.y, 0)
    self.y = min(self.y, self.world_height)
end

update! (generic function with 3 methods)

In [121]:
%%julia
function main()
    kwargs = (
        world_width = WORLD_WIDTH,
        world_height = WORLD_HEIGHT,
    )
    preys = [Prey(; kwargs...) for i in 1:10]
    predators = [Predator(; kwargs...) for i in 1:10]
    plants = [Plant(; kwargs...) for i in 1:100]

    timestep = 0
    while timestep < TIMESTEPS
        # update all agents
        #[f.update([]) for f in plants]  # no need to update the plants; they do not move
        for a in preys
            update!(a, plants)
        end
        for a in predators
            update!(a, preys)
        end

        # handle eaten and create new plant
        # plants = [p for p in plants if p.is_alive]
        filter!(p -> p.is_alive, plants)
        append!(plants, [Plant(; kwargs...) for i in 1:2])

        # handle eaten and create new preys
        # preys = [p for p in preys if p.is_alive]
        filter!(p -> p.is_alive, preys)
        for p in preys
            if p.energy > 5
                p.energy = 0
                push!(preys, Prey(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # handle old and create new predators
        # predators = [p for p in predators if p.age < 2000]
        filter!(p -> p.age < 2000, predators)
        for p in predators
            if p.energy > 10
                p.energy = 0
                push!(predators, Predator(; x = p.x + rand(-20:20), y = p.y + rand(-20:20), kwargs...))
            end
        end

        # write data to output file
        #[print(timestep, ',', 'Position', ',', 'Predator', ',', a.x, ',', a.y, file=f) for a in predators]
        #[print(timestep, ',', 'Position',  ',', 'Prey', ',', a.x, ',', a.y, file=f) for a in preys]
        #[print(timestep, ',', 'Position',  ',', 'Plant', ',', a.x, ',', a.y, file=f) for a in plants]

        timestep += 1
    end
    println("$(length(predators)), $(length(preys)), $(length(plants))")
end

main (generic function with 1 method)

In [122]:
%julia using BenchmarkTools
%julia @btime main()

33, 477, 341
5, 1487, 244
10, 1249, 322
0, 2378, 242
28, 658, 285
18, 1827, 309
23, 1297, 261
34, 817, 308
31, 433, 293
41, 553, 344
16, 1357, 276
14, 1041, 230
29, 1351, 238
  834.828 ms (524020 allocations: 132.59 MiB)


In [123]:
%julia @benchmark main()

0, 2419, 262
13, 1183, 222
6, 1759, 291
19, 1540, 370
22, 931, 317
17, 991, 296
26, 1185, 249
23, 844, 290
14, 1120, 247
0, 2406, 288
38, 514, 314
20, 1052, 325


BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  986.515 ms …    2.543 s  ┊ GC (min … max): 2.20% … 0.61%
 Time  (median):        1.397 s               ┊ GC (median):    1.33%
 Time  (mean ± σ):      1.581 s ± 671.625 ms  ┊ GC (mean ± σ):  3.34% ± 5.29%

  █            █    █                                         █  
  █▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  987 ms           Histogram: frequency by time          2.54 s <

 Memory estimate: 144.49 MiB, allocs estimate: 1954947.

In [124]:
%julia TIMESTEPS

10000